# 🖼️ DWT Loss Scaling Test on Real Image

This notebook tests the automatic DWT loss scaling on a real image to demonstrate how it works.


In [ ]:
import torch
import torch.nn.functional as F
import sys
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Add current directory to path (Jupyter notebook version)
import os
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

from utils.loss_utils import get_dwt_subbands, charbonnier_loss, DWTLossScaler


In [ ]:
# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load image (change this path to your image)
image_path = "/content/llff/nerf_llff_data/fern/images/IMG_4026.JPG"

# If image doesn't exist, create a synthetic one
if not os.path.exists(image_path):
    print(f"Image not found: {image_path}")
    print("Creating synthetic test image...")
    
    # Create synthetic image
    synthetic_img = torch.randn(1, 3, 512, 512)
    synthetic_img = torch.clamp(synthetic_img, 0, 1)
    
    # Save as temporary image
    img_array = (synthetic_img.squeeze(0).permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
    temp_path = "temp_test_image.jpg"
    Image.fromarray(img_array).save(temp_path)
    image_path = temp_path
    print(f"Created synthetic image: {temp_path}")

print(f"Loading image: {image_path}")


In [ ]:
def load_and_preprocess_image(image_path, target_size=(512, 512)):
    """Load and preprocess image for testing."""
    try:
        # Load image
        img = Image.open(image_path).convert('RGB')
        print(f"Original image size: {img.size}")
        
        # Resize to target size
        img = img.resize(target_size, Image.LANCZOS)
        
        # Convert to tensor and normalize to [0, 1]
        img_array = np.array(img).astype(np.float32) / 255.0
        
        # Convert to PyTorch tensor: (H, W, C) -> (1, C, H, W)
        img_tensor = torch.from_numpy(img_array).permute(2, 0, 1).unsqueeze(0)
        
        return img_tensor, img.size
    except Exception as e:
        print(f"Error loading image: {e}")
        return None, None

# Load the image
img_tensor, original_size = load_and_preprocess_image(image_path)

if img_tensor is None:
    print("❌ Failed to load image")
else:
    # Move to device
    img_tensor = img_tensor.to(device)
    print(f"Image tensor shape: {img_tensor.shape}")
    print(f"Image value range: [{img_tensor.min():.3f}, {img_tensor.max():.3f}]")


In [ ]:
# Create a slightly different version (simulating prediction vs ground truth)
noise = torch.randn_like(img_tensor) * 0.1  # 10% noise
pred_tensor = torch.clamp(img_tensor + noise, 0, 1)

print("🔍 Computing DWT subbands...")

# Get DWT subbands for both images
pred_bands = get_dwt_subbands(pred_tensor)
gt_bands = get_dwt_subbands(img_tensor)

print("✅ DWT subbands computed successfully!")


In [ ]:
# Show subband information
print("📊 DWT Subband Information:")
for name in ['LL1', 'LH1', 'HL1', 'HH1', 'LL2', 'LH2', 'HL2', 'HH2']:
    pred_band = pred_bands[name]
    gt_band = gt_bands[name]
    print(f"  {name}: {pred_band.shape}, range: [{pred_band.min():.3f}, {pred_band.max():.3f}]")


In [ ]:
# Compute DWT losses (using your actual weights)
print("🧮 Computing DWT losses with your weights...")

# Level 1 losses
ll1_loss = charbonnier_loss(pred_bands['LL1'], gt_bands['LL1'])
lh1_loss = charbonnier_loss(pred_bands['LH1'], gt_bands['LH1'])
hl1_loss = charbonnier_loss(pred_bands['HL1'], gt_bands['HL1'])
hh1_loss = charbonnier_loss(pred_bands['HH1'], gt_bands['HH1'])

# Level 2 losses
ll2_loss = charbonnier_loss(pred_bands['LL2'], gt_bands['LL2'])
lh2_loss = charbonnier_loss(pred_bands['LH2'], gt_bands['LH2'])
hl2_loss = charbonnier_loss(pred_bands['HL2'], gt_bands['HL2'])
hh2_loss = charbonnier_loss(pred_bands['HH2'], gt_bands['HH2'])

print("Individual subband losses:")
print(f"  LL1: {ll1_loss.item():.6f}")
print(f"  LH1: {lh1_loss.item():.6f}")
print(f"  HL1: {hl1_loss.item():.6f}")
print(f"  HH1: {hh1_loss.item():.6f}")
print(f"  LL2: {ll2_loss.item():.6f}")
print(f"  LH2: {lh2_loss.item():.6f}")
print(f"  HL2: {hl2_loss.item():.6f}")
print(f"  HH2: {hh2_loss.item():.6f}")

# Compute total DWT loss (with your weights: LL1=1.0, LL2=0.5, others=0.0)
dwt_loss = 1.0 * ll1_loss + 0.5 * ll2_loss
print(f"\nTotal DWT loss (LL1 + 0.5×LL2): {dwt_loss.item():.6f}")


In [ ]:
def load_and_preprocess_image(image_path, target_size=(512, 512)):
    """Load and preprocess image for testing."""
    try:
        # Load image
        img = Image.open(image_path).convert('RGB')
        print(f"Original image size: {img.size}")
        
        # Resize to target size
        img = img.resize(target_size, Image.LANCZOS)
        
        # Convert to tensor and normalize to [0, 1]
        img_array = np.array(img).astype(np.float32) / 255.0
        
        # Convert to PyTorch tensor: (H, W, C) -> (1, C, H, W)
        img_tensor = torch.from_numpy(img_array).permute(2, 0, 1).unsqueeze(0)
        
        return img_tensor, img.size
    except Exception as e:
        print(f"Error loading image: {e}")
        return None, None
